In [17]:
from sklearn.model_selection import train_test_split

from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Conv2D, Dense, Lambda
from tensorflow.keras.utils import to_categorical
import tensorflow
import pandas as pd
import numpy as np

In [18]:
normal_model = load_model("my_model_1.h5")
test = pd.read_csv('./input/train.csv')
print("Data are Ready!!")
print(f"Testing data size is {test.shape}")

Data are Ready!!
Testing data size is (42000, 785)


# 테스트 데이터 전처리

In [19]:
X_test = test.drop(['label'], 1).values
y_test = test['label'].values

_, X_test, _, y_test = train_test_split(X_test, y_test, test_size=0.1, random_state=0)

X_test = X_test / 255.0
X_test = X_test.reshape(-1,28,28,1)

y_test = to_categorical(y_test)

print(f"Label size {y_test.shape}")

/var/folders/v3/ybc9hrrd0yg6d_zr2sbckrtm0000gn/T/ipykernel_6317/2536502158.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  X_test = test.drop(['label'], 1).values


Label size (4200, 10)


# 노이즈 발생 테스트

In [20]:
random_array = np.random.choice([1.01, 0.99], size=(10,10), p=[0.5, 0.5])

print(random_array)

[[1.01 1.01 0.99 1.01 1.01 1.01 0.99 1.01 1.01 1.01]
 [1.01 0.99 1.01 0.99 0.99 0.99 0.99 1.01 1.01 0.99]
 [1.01 1.01 1.01 0.99 0.99 1.01 1.01 1.01 1.01 1.01]
 [0.99 1.01 1.01 1.01 0.99 0.99 0.99 1.01 0.99 1.01]
 [0.99 0.99 0.99 0.99 0.99 0.99 0.99 0.99 1.01 1.01]
 [1.01 0.99 1.01 1.01 1.01 1.01 0.99 0.99 0.99 1.01]
 [0.99 1.01 1.01 1.01 0.99 1.01 1.01 0.99 0.99 1.01]
 [1.01 1.01 1.01 1.01 1.01 0.99 1.01 0.99 0.99 1.01]
 [1.01 1.01 1.01 1.01 1.01 1.01 0.99 1.01 1.01 1.01]
 [1.01 0.99 1.01 0.99 0.99 1.01 0.99 1.01 0.99 0.99]]


# 레이어 -> 다음 레이어로 넘어가는 상황에서 노이즈 발생 (합성곱 이후)

In [21]:
def random_multiplier(shape):
    random_factors = np.random.choice([1.01, 0.99], size=shape, p=[0.5, 0.5])
    return random_factors

class RandomMultiplierLayer(tensorflow.keras.layers.Layer):
    def call(self, inputs):
        random_factors = random_multiplier(inputs.shape[-1])
        return inputs * random_factors

# 합성곱 과정에 노이즈 발생(가중치, 편향에 노이즈)

In [22]:
def simulate_gaussian_noise(model):
    for layer in model.layers:
        if isinstance(layer, (tensorflow.keras.layers.Conv2D, tensorflow.keras.layers.Dense)):
            weights = layer.get_weights()[0]
            biases = layer.get_weights()[1]

            noisy_weights = weights * np.random.choice([1.02, 0.98], size=weights.shape, p=[0.5, 0.5])
            noisy_biases = biases * np.random.choice([1.02, 0.98], size=biases.shape, p=[0.5, 0.5])
            layer.set_weights([noisy_weights, noisy_biases])

# !! 테스트 !!

# 100번 테스트 돌려보기

In [23]:
cycle = 100
for _ in range(cycle):
    # error_model = Sequential()
    # for layer in normal_model.layers:
    #     error_model.add(layer)  # 기존 레이어 추가
    #     if isinstance(layer, (tensorflow.keras.layers.Conv2D, tensorflow.keras.layers.Dense)):
    #         error_model.add(RandomMultiplierLayer())  # 랜덤한 값을 곱하는 레이어 추가

    # # 컴파일 및 요약
    # error_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    error_model = normal_model
    simulate_gaussian_noise(error_model) # 가중치에도 노이즈 끼길 원하는 경우
    normal_model = load_model("my_model_1.h5")
    test_error_loss, test_error_accuracy = error_model.evaluate(X_test, y_test, verbose=2)
    test_normal_loss, test_normal_accuracy = normal_model.evaluate(X_test, y_test, verbose=2)
    print(f"노이즈가 없는 경우: {test_normal_accuracy} \n노이즈가 있는 경우: {test_error_accuracy} \n차이값: {test_normal_accuracy-test_error_accuracy}")

2023-08-10 23:00:01.527896: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


132/132 - 2s - loss: 0.0501 - accuracy: 0.9860 - 2s/epoch - 18ms/step


2023-08-10 23:00:03.864397: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


132/132 - 2s - loss: 0.0496 - accuracy: 0.9860 - 2s/epoch - 16ms/step
노이즈가 없는 경우: 0.9859523773193359 
노이즈가 있는 경우: 0.9859523773193359 
차이값: 0.0
132/132 - 2s - loss: 0.0497 - accuracy: 0.9855 - 2s/epoch - 12ms/step


2023-08-10 23:00:07.741730: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


...

132/132 - 2s - loss: 0.0496 - accuracy: 0.9860 - 2s/epoch - 16ms/step
노이즈가 없는 경우: 0.9859523773193359 
노이즈가 있는 경우: 0.9854761958122253 
차이값: 0.0004761815071105957
132/132 - 2s - loss: 0.0501 - accuracy: 0.9855 - 2s/epoch - 12ms/step


2023-08-10 23:00:27.999989: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


132/132 - 2s - loss: 0.0496 - accuracy: 0.9860 - 2s/epoch - 15ms/step
노이즈가 없는 경우: 0.9859523773193359 
노이즈가 있는 경우: 0.9854761958122253 
차이값: 0.0004761815071105957
132/132 - 2s - loss: 0.0494 - accuracy: 0.9857 - 2s/epoch - 12ms/step


2023-08-10 23:00:31.901394: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


KeyboardInterrupt: 

# 가중치에 노이즈를 더한 경우 가중치 확인용

In [14]:
def save_weights_to_text(model, file_path):
    with open(file_path, 'w') as file:
        for layer in model.layers:
            weights = layer.get_weights()
            if len(weights) > 0:
                weights_text = [str(w.tolist()) for w in weights]
                weights_str = "\n".join(weights_text)
                file.write(f"Layer: {layer.name}\n")
                file.write(weights_str + "\n\n")

In [15]:
weights_file_path = './model_weights.txt'
save_weights_to_text(normal_model, weights_file_path)

In [16]:
weights_file_path = './error_model_weights.txt'
save_weights_to_text(error_model, weights_file_path)